## Packages

In [31]:
#setting up my usual packages
from __future__ import print_function, division
import os
import numpy as np
import pandas as pd
import matplotlib as plt
import pylab as pl
import scipy as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns;

try: 
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Download/Read in files

In [2]:
PUIdata = os.getenv('PUIDATA')
print (PUIdata)

/home/cusp/rxl204/PUIdata


#### State data source in markdown

In [3]:
#download files using curl
if not os.path.isfile(PUIdata + "/BORO_zip_files_csv/MN.csv"):
    print('Downloading...')
    os.system('curl -O https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_16v2%20.zip')
    os.system('mv nyc_pluto_16v2%20.zip ' + PUIdata)
    os.system('unzip ' + PUIdata + '/nyc_pluto_16v2%20.zip -d ' + PUIdata)
    if os.path.isfile(PUIdata + "/BORO_zip_files_csv/MN.csv"):
        print('File in place, proceed!')
else:
    print('File in place, proceed!')

File in place, proceed!


In [4]:
#using wget (web get)
url = 'https://data.cityofnewyork.us/api/views/rgfe-8y2z/rows.csv?accessType=DOWNLOAD'
filename = 'LL84_2013.csv'

if not os.path.isfile(PUIdata + '/' + filename):
    print('Downloading...')
    os.system('wget ' + url)
    os.system('mv rows.csv?accessType=DOWNLOAD ' + filename)
    os.system('mv ' + filename + ' ' + PUIdata)
    if os.path.isfile(PUIdata + '/' + filename):
        print('File in place, proceed!')
else:
    print('File in place, proceed!')

File in place, proceed!


In [5]:
#read files
dfman = pd.read_csv(PUIdata + '/BORO_zip_files_csv/MN.csv')

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19,22,23,25,63,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def getCitiBikeCSV(datestring):
    print ("Downloading", datestring)
    ### First I will check that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip")
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
            os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")


In [7]:
datestring = '201503'
getCitiBikeCSV(datestring)

file in place, you can continue


In [8]:
#opening files from cusp data facility
DFdata = os.getenv("DFDATA")
DFdata

'/gws/open/NYCOpenData/nycopendata/data'

In [9]:
#check path from terminal
data = pd.read_csv(DFdata + '/j34j-vqvt/1409875200/j34j-vqvt.csv')
data.head()

,YEARMONTH,INCIDENTCLASSIFICATION,INCIDENTBOROUGH,INCIDENTCOUNT,AVERAGERESPONSETIME
0,NaN,INCIDENTCLASSIFICATION,INCIDENTBOROUGH,NaN,AVERAGERESPONSETIME
1,200907.0,Structural Fires,Citywide,1947.0,3:54
2,200907.0,Structural Fires,Manhattan,435.0,4:00
3,200907.0,Structural Fires,Bronx,432.0,3:59
4,200907.0,Structural Fires,Staten Island,90.0,4:34


In [11]:
#check for columns with null values
data.isnull().any()

YEARMONTH                  True
INCIDENTCLASSIFICATION    False
INCIDENTBOROUGH           False
INCIDENTCOUNT              True
AVERAGERESPONSETIME       False
dtype: bool

In [12]:
#check number of null values per column
data.isnull().sum()

YEARMONTH                 43
INCIDENTCLASSIFICATION     0
INCIDENTBOROUGH            0
INCIDENTCOUNT              1
AVERAGERESPONSETIME        0
dtype: int64

In [15]:
df = data.dropna()
df.isnull().any()

YEARMONTH                 False
INCIDENTCLASSIFICATION    False
INCIDENTBOROUGH           False
INCIDENTCOUNT             False
AVERAGERESPONSETIME       False
dtype: bool

In [17]:
#check dtype
df.dtypes

YEARMONTH                 float64
INCIDENTCLASSIFICATION     object
INCIDENTBOROUGH            object
INCIDENTCOUNT             float64
AVERAGERESPONSETIME        object
dtype: object

In [21]:
#change to int
df['YEARMONTH'] = df['YEARMONTH'].astype(int)
df['INCIDENTCOUNT'] = df['INCIDENTCOUNT'].astype(int)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


## FIGURE OUT DATETIME !!!!

In [22]:
pd.DataFrame({
    'YEAR': pd.to_datetime(df['YEARMONTH']).dt.year,
    'MONTH': pd.to_datetime(df['YEARMONTH']).dt.month})
df.head()

,YEARMONTH,INCIDENTCLASSIFICATION,INCIDENTBOROUGH,INCIDENTCOUNT,AVERAGERESPONSETIME
1,200907,Structural Fires,Citywide,1947,3:54
2,200907,Structural Fires,Manhattan,435,4:00
3,200907,Structural Fires,Bronx,432,3:59
4,200907,Structural Fires,Staten Island,90,4:34
5,200907,Structural Fires,Brooklyn,652,3:29


### merge df

DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False)[source]

A.merge(B, left_on='lkey', right_on='rkey', how='outer')

In [ ]:
#rows that were lost in the merge
ms[~ms['Country'].isin(df_['Country'])]

https://github.com/fedhere/PUI2017_fb55/blob/master/HW5_fb55/Assignment3_solution.ipynb

In [ ]:
df['haircolor'] = df['haircolor'].replace({'Brown':1, 'Blonde':2, 'Black':3})

In [ ]:
#converting columns to numeric values and throwing away 
s = dfen['Site EUI(kBtu/ft2)'] #extract Site EUI as series
ss = pd.to_numeric(s, errors='coerce')  # convert strings to float, use errors=coerce to set missing values to nan
dfen['Site EUI(kBtu/ft2)'] = ss
##check that your conversion worked: e.g.
print (dfen['Site EUI(kBtu/ft2)'].astype(float))

In [ ]:
#add reading JSON
import sys
import json
try:
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib


def get_jsonparsed_data(url):
    """
    from http://stackoverflow.com/questions/12965203/how-to-get-json-from-webpage-into-python-script
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urllib.urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)
jsonData = get_jsonparsed_data(\
                    "https://serv.cusp.nyu.edu/~fbianco/PUI2017/data/cityWeather.json")
jsonData

In [ ]:
#add reading API

#Read MTA API

if not len(sys.argv) == 4:
	print ("Invalid number of arguments. Run as: python show_bus_locations_rxl204.py <MTA_KEY> <BUS_LINE> <BUS_LINE.csv>")
	sys.exit()

url = "http://bustime.mta.info/api/siri/vehicle-monitoring.json?key=" + sys.argv[1] + "&VehicleMonitoringDetailLevel=calls&LineRef=" + sys.argv[2]

response = urllib.urlopen(url)
data = response.read().decode("utf-8")
data = json.loads(data)

#Opens a file for writing
fout = open(sys.argv[3],"w")
fout.write("Latitude,Longitude,Stop Name,Stop Status\n")

Bus_Count = len(data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'])

for i in (range(0,Bus_Count)):
	Bus_Lat = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['VehicleLocation']['Latitude']
	Bus_Long = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['VehicleLocation']['Longitude']
	if data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['OnwardCalls']=={}:
		Stop_Name='NA'
	else: 
		Stop_Name = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['OnwardCalls']['OnwardCall'][0]['StopPointName']
	if data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['OnwardCalls']=={}:
		Stop_Status='NA'
	else: 
		Stop_Status = data['Siri']['ServiceDelivery']['VehicleMonitoringDelivery'][0]['VehicleActivity'][i]['MonitoredVehicleJourney']['OnwardCalls']['OnwardCall'][0]['Extensions']['Distances']['PresentableDistance'] 

	fout.write("%s,%s,%s,%s\n" %(Bus_Lat, Bus_Long, Stop_Name, Stop_Status))


## Hypothesis Testing

Stating the Null Hypothesis: The bus rerouting does not change the average travel time.

$H_0$: TimeNew.mean() >= TimeOld.mean()

$H_a$: TimeNew.mean() < TimeOld.mean()

$\alpha=0.05$

Z = \frac{\mu_{pop} - \mu_{sample}}{\sigma / \sqrt{N}}

In [ ]:
#calculating z-scores
# here it is ok to use either the stdev from timesOld or from timesNew
z = (timesOldMean - timesNew.mean()) / (timesOldStd / np.sqrt(len(timesNew)))
print("The Z statistics is Z = {0:.2f}".format(Z))

Z score for Bus X8 rerouting: 2.6
This means we are 2.6 standard deviations away
from the mean of the old trip duration

The Null Hypothesis that the new route makes commuting time worse
IS REJECTED at the 95.0% significance level (p<0.05)

#### Deciding on statistical test
2 samples, categorical data
TWO OPTIONS z test, or chi square test.

### Z TEST
the z test compares the standard deviation of the expected distribution and the observed result. it tells you literally how many standard deviations from the tail an observation is, under the _assumption of normality
must define the sample standard deviation
standard deviation of the sampling distribution the distribution is Binomial, the binomail stdev is
(see a proof here!: http://stats.stackexchange.com/questions/29641/standard-error-for-the-mean-of-a-sample-of-binomial-random-variables!):
$\sqrt{\frac{p(1 - p)}{n}}$
for 2 samples this becomes
$\sqrt{ \frac{ \hat{p}(1 - \hat{p})} {n1} + \frac{ \hat{p}(1 - \hat{p})} {n1} }$

Z-tests are statistical calculations that can be used to compare population means to a sample's. The z-score tells you how far, in standard deviations, a data point is from the mean or average of a data set. A z-test compares a sample to a defined population and is typically used for dealing with problems relating to large samples (n > 30). Z-tests can also be helpful when we want to test a hypothesis. Generally, they are most useful when the standard deviation is known.

In [ ]:
ztest = (tOldMean -tNew.mean())/(tOldStdev/len(df)**0.5)
print (ztest)
#population mean - sample mean / std/sqrt of pop

Z score for Bus X8 rerouting: 2.6
This means we are 2.6 standard deviations away
from the mean of the old trip duration


The Null Hypothesis that the new route makes commuting time worse
IS REJECTED at the 95.0% significance level (p<0.05)

In [26]:
alpha = 0.05
#we like fractions better then percentages. as a rule of thumb, either use fractions or counts
P_0 = 3.5 * 0.01 
P_1 = 70.1 * 0.01

n_0 = 409
n_1 = 564

#lets get the counts by multiplying by the sample size
Nt_0 = P_0 * n_0
Nt_1 = P_1 * n_1

#depending on distribution, calculate standard error
sp_stdev = lambda p, n: np.sqrt( p * ( 1 - p ) / n[0] +  p * ( 1 - p ) / n[1]  )
sp_stdev_2y = sp_stdev((Nt_0 + Nt_1) / (n_0 + n_1), [n_0, n_1])
print ( "test standard deviation error: %.3f"%sp_stdev_2y)

test standard deviation error: 0.032


In [28]:
#calculate z-score
zscore = lambda p0, p1, s : (p0 - p1) / s
z_2y = zscore(P_1, P_0, sp_stdev_2y)
print ('z score for 1-3 years "ever employed in CEO jobs": z = %.3f'%z_2y)

z score for 1-3 years "ever employed in CEO jobs": z = 20.770


if $p&lt;\alpha$ : reject H0 **
IMPORTANT!! note that this P in the bottom line of the table is not the p-value, but
p-value = 1-P

Since p_2y = 1 - 0.9998 = 0.00 is smaller than the critical value of 0.05, the null hypothesis is rejected. 

#get the corresponding p-value by looking up the table: z score of 0.83-> 0.7967 

1-0.7967 > 0.05

Since the p value of 0.2 is not smaller than the critical value 0.05, the null hypothesis is not rejected. 


## T test
t-tests are calculations used to test a hypothesis, but they are most useful when we need to determine if there is a statistically significant difference between two independent sample groups. In other words, a t-test asks whether a difference between the means of two groups is unlikely to have occurred because of random chance. Usually, t-tests are most appropriate when dealing with problems with a limited sample size (n < 30).

## Chi Square Test 
$\chi^2$ test
the chisq statistics tests a number against the distribution of the following quantity:
$$\chi^2 = \Sigma \frac{(observation - expectation)^2}{expectation}$$
if we talk about sample fractions that is
$$\chi^2 = \Sigma \frac{(f_{observed} - f_{expectated})^2}{f_{expected}}$$
turns out this quantity is distributed according to a chi square distribution, so if i get the $\chi^2$ statistics i can compare it to the full chisq distribution and see how far in the tail it is

In [32]:
from __future__ import print_function, division
import numpy as np


def evalChisq(values):
    '''Evaluates the chi sq from a contingency value
    Arguments:
    values: 2x2 array or list, the contingengy table
    '''
    if not (len(values.shape) == 2 and values.shape == (2,2)):
        print ("must pass a 2D array")
        return -1
    values = np.array(values)
    E = np.empty_like(values)
    for j in range(len(values[0])):
        for i in range(2):
            
            E[i][j] = ((values[i,:].sum() * values[:,j].sum()) / 
                        (values).sum())
    return ((values - E)**2 / E).sum()

In [33]:
sample_values_ceojob  = np.array([[0.701 * 564, 0.299 * 564], [0.0305 * 409, 0.965 * 409]])

chisq_ceojob = evalChisq(sample_values_ceojob)
DOF = len(sample_values_ceojob) - 1
print ('chi sq statistics for "ever employed in CEO jobs": '+
       'chisq = {:.3f}, DOF = {:d}'.format(chisq_ceojob, DOF))

chi sq statistics for "ever employed in CEO jobs": chisq = 436.223, DOF = 1


In [34]:
chimin_alpha5pc = 3.84
print ("The Null hypothesis that the program does not affect CEO employability, ")
print ("measured as: the inmates not participating in the program are employed in CEO jobs ")
print ("in an equal or higher percentage within 1-3 years of release")
if chisq_ceojob > chimin_alpha5pc :
    print ("can be rejected at alpha = 0.05")
else: 
    print ("cannot be rejected (p<0.05)")
print ("with a chi square statistics of %.2f"%chisq_ceojob )

The Null hypothesis that the program does not affect CEO employability, 
measured as: the inmates not participating in the program are employed in CEO jobs 
in an equal or higher percentage within 1-3 years of release
can be rejected at alpha = 0.05
with a chi square statistics of 436.22


Since the chisquare statistic of 436.22 is more than the 3.84, the null hypothesis can be rejected at alpha = 0.05. 

## KS tests to compare 2 samples
http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ks_2samp.html

scipy.stats.ks_2samp(data1, data2)[source]

Null: the two distribution come from the same parent distribution

In [ ]:
ks = scipy.stats.ks_2samp(df.dayduration, df.nightduration)
print (ks)
if ks[1] >= alpha: 
    print ("The Null hypothesis that the two samples are generated " + 
            "by the same parent distribution cannot be rejected (p>=0.05)")
else: 
    print ("The Null hypothesis that the two samples are generated " + 
            "by the same parent distribution is rejected (p<0.05)")

## Correlation
Now retest using a test for correlation.
That will answer a slightly different question though - formulate the NULL appropriately. The tests for correlations (generally) requires the variable to be paired, so that I can tell if x changes does y change similarly. But the datasets are of different size! You will need to reduce them to the same size. You can do that by subsampling of the data: take only 1 ride every of 200, which you can achieve "slicing and broadcasting" the array or using one of the python function (built in python numpy.random.choice() functions for example: Docstring: choice(a, size=None, replace=True, p=None)
Generates a random sample from a given 1-D array
.. versionadded:: 1.7.0

Parameters ...
But make sure you understand how to use it! there is an option "replace" which you should think about. Pearson's test for correlation¶
notice that the Pearson's is a pairwise test: the samples need to be a. the same size b. sorted! (how??)
http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr
Hypothesis
H0: There is no strong relationship between trip duration for day and night riders
Pearson's test for correlation
scipy.stats.pearsonr(x, y)



In [ ]:
#more rides during the day than during the night: subsample day rides
dfday = cb2015[cb2015.dayride].iloc[np.random.choice(range(len(cb2015[cb2015.dayride])), (~cb2015.dayride).sum(), 
                                      replace=False)]

#or

df['dayduration'].dropna(inplace= True)
df['nightduration'].dropna(inplace= True)

#select one trip from every 200
df3 = df[df.index % 100 == 0]
df3.head()

X = df3.dayduration
Y = df3.nightduration

pr = scipy.stats.pearsonr(X, Y)
print (pr)

The results from Pearson's correlation indicates that there is no correlation in trip durations between daytime and nighttime riders.
Pearson's correlation coefficient between two variables is defined as the covariance of the two variables divided by the product of their standard deviations.

In [ ]:
spr = scipy.stats.spearmanr(X, Y)
print(spr)

Spearman's r test result SpearmanrResult(correlation=0.99999921875810915, pvalue=0.0)
The Null hypothesis that the two samples are generated from uncorrelated distributions. is rejected (p = 0.000<0.05)
This test is often used to assess if sample are generated by the same distribution but in fact it tests if the samples are generated by correlated distributions. As a test to assess independence it is not powerful, as even a small amount of correlation would raise the p-value

## OLS 

In [ ]:
smf.ols(formula = 'BBL ~ Postcode + Longitude + I(Postcode**2)', data = test3.fit(). summary()
#I dimensionality matrix
        
lm = smf.ols(formula='y ~ x', data=dfjoin).fit()
lm.summary()

Plotting matplotlib: 
        https://www.datascience.com/learn-data-science/tutorials/creating-data-visualizations-matplotlib-data-science-python  
        

## Plotting Error Bars

The Scatter Plots with Error Bars procedure extends the capability of the basic scatter plot by allowing you to plot
the variability in Y and X corresponding to each point. Each point on the plot represents the mean or median of
one or more values for Y and X within a subgroup. 

The error bars may represent the standard
deviation (SD) of the data, the standard error of the mean (SE), a confidence interval, the data range, or
percentiles.

Axes.errorbar(x, y, yerr=None, xerr=None, fmt='', ecolor=None, elinewidth=None, capsize=None, barsabove=False, lolims=False, uplims=False, xlolims=False, xuplims=False, errorevery=1, capthick=None, *, data=None, **kwargs)

In [ ]:
#plotting scatter with error bars
fig, ax = plt.subplots()

scatter = ax.scatter(df_.gdp * 1e-12, df_['Number of mass shootings'])
ax.errorbar(df_.gdp * 1e-12, df_['Number of mass shootings'], 
            yerr = np.sqrt(df_['Number of mass shootings'] * 1.0), fmt = '.')
ax.set_xlabel("GDP (in tillion $)")
ax.set_ylabel("Number of mass shooting per person")
ax.set_ylim();



https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.linspace.html

In [ ]:
## Plotting 
scatter plots 
zoomed scatter
histogram 
bar plot 
seaborn scatter matrix
line fitting plots (models) 

## Modeling
fit a line to the Number of mass shootings per person as a function of Average total all civilian firearms per person. Before you do it it is a good idea to make sure that both quantities are roughly of the same order of magnitude (unity). Use

In [ ]:
df_['x'] = df_[u'Average total all civilian firearms'] / df_['pop'] * 10
df_['y'] = df_[u'Number of mass shootings'] / df_['pop'] * 1e7

## Pearson Correlation
scipy.stats.pearsonr(x, y)[source]
Calculates a Pearson correlation coefficient and the p-value for testing non-correlation.

The Pearson correlation coefficient measures the linear relationship between two datasets. Strictly speaking, Pearson’s correlation requires that each dataset be normally distributed. Like other correlation coefficients, this one varies between -1 and +1 with 0 implying no correlation. Correlations of -1 or +1 imply an exact linear relationship. Positive correlations imply that as x increases, so does y. Negative correlations imply that as x increases, y decreases.

The p-value roughly indicates the probability of an uncorrelated system producing datasets that have a Pearson correlation at least as extreme as the one computed from these datasets. The p-values are not entirely reliable but are probably reasonable for datasets larger than 500 or so.

In [ ]:
manhattan = cb2015[cb2015.M].iloc[np.random.choice(range(len(cb2015[cb2015.M])), (cb2015.BK).sum(), 
                                      replace=False)]
parsons = scipy.stats.pearsonr(np.sort(manhattan.tripduration), np.sort(cb2015[cb2015.BK].tripduration))
print ("Parson's test result", parsons)
if parsons[1] >= alpha: 
    print (r"The Null hypothesis that the two samples are generated from uncorrelated distributions. " + 
           "cannot be rejected (p>=0.05)")
else: 
    print ("The Null hypothesis that the two samples are generated from uncorrelated distributions. " + 
           "is rejected (p = %.3f<0.05)"%parsons[1])


## Spearman Correlation

scipy.stats.spearmanr(a, b=None, axis=0)[source]
Calculates a Spearman rank-order correlation coefficient and the p-value to test for non-correlation.

The Spearman correlation is a nonparametric measure of the monotonicity of the relationship between two datasets. Unlike the Pearson correlation, the Spearman correlation does not assume that both datasets are normally distributed. Like other correlation coefficients, this one varies between -1 and +1 with 0 implying no correlation. Correlations of -1 or +1 imply an exact monotonic relationship. Positive correlations imply that as x increases, so does y. Negative correlations imply that as x increases, y decreases.

The p-value roughly indicates the probability of an uncorrelated system producing datasets that have a Spearman correlation at least as extreme as the one computed from these datasets. The p-values are not entirely reliable but are probably reasonable for datasets larger than 500 or so

In [ ]:
beta1 = cor[0] * df_.y.std() / df_.x.std()
beta0 = df_.y.mean() - beta1 * df_.x.mean()

fig = pl.figure(figsize=(10,5))
ax = fig.add_subplot(111)
df_.plot.scatter(x="x", y="y", ax=ax)
ax.plot(df_.x, beta0 + beta1 * df_.x, '-', color='k', label="analytical")
ax.set_ylabel("Number of mass shootings per person (x 10,000,000)")
ax.set_xlabel("Average civilian fire arms per person")
pl.legend()
print ("coefficients from analytical solution", beta0, beta1)

## Polyfit 
numpy.polyfit(x, y, deg, rcond=None, full=False, w=None, cov=False)[source]
Least squares polynomial fit.

Fit a polynomial p(x) = p[0] * x**deg + ... + p[deg] of degree deg to points (x, y). Returns a vector of coefficients p that minimises the squared error.

In [ ]:
polyfit_coeffs = np.polyfit(df_['x'], df_['y'], 1)

fig = pl.figure(figsize=(10,5))
ax = fig.add_subplot(111)
df_.plot.scatter(x="x", y="y", ax=ax)
ax.plot(df_.x, polyfit_coeffs[0] + polyfit_coeffs[1] * df_.x, '-', color='DarkOrange', label="polyfit")
ax.set_ylabel("Number of mass shootings per person (x 10,000,000)")
ax.set_xlabel("Average civilian fire arms per person")
pl.legend()
print ("polyfit coefficients:", polyfit_coeffs)

## Statsmodels OLS


In [ ]:
modelOLS = st.OLS(exog=st.add_constant(df_.x), endog=df_.y).fit()

fig = pl.figure(figsize=(10,5))
ax = fig.add_subplot(111)
df_.plot.scatter(x="x", y="y", ax=ax)
ax.plot(df_.x, polyfit_coeffs[1] + polyfit_coeffs[0] * df_.x, '-', color='DarkOrange', label="polyfit")
ax.plot(df_.x, modelOLS.fittedvalues, color="DarkGreen", label="OLS")
ax.set_ylabel("Number of mass shootings per person (x 10,000,000)")
ax.set_xlabel("Average civilian fire arms per person")
pl.legend();

## Statsmodel WLS
Note: when the number of mass shootings is 0 the error is 0. But 0 errros cannot be used as weight (cause 1/0 = infinity). Thus I am setting the error on the values Nmassshootings = 0 to 1, which is the uncertainty on Nmassshootings = 1.


In [ ]:
df_['yerr'] = np.sqrt(df_[u'Number of mass shootings'] 
                             / df_['pop'] * 1e7) + 1
df_.w = 1.0 / df_.yerr


modelWLS = st.WLS(exog=st.add_constant(df_.x), endog=df_.y,
      weights = 1.0/df_.w).fit()

fig = pl.figure(figsize=(10,5))
ax = fig.add_subplot(111)
df_.plot.scatter(x="x", y="y", yerr=df_.yerr, ax=ax)
ax.plot(df_.x, polyfit_coeffs[1] + polyfit_coeffs[0] * df_.x, '-', color='DarkOrange', label="polyfit")
ax.plot(df_.x, modelOLS.fittedvalues, color="DarkGreen", label="OLS")
ax.plot(df_.x, modelWLS.fittedvalues, '-', color='IndianRed', label="WLS")
ax.set_ylabel("Number of mass shootings per person (x 10,000,000)")
ax.set_xlabel("Average civilian fire arms per person")
pl.legend()
print ("WLS coefficients:",  modelWLS.params)

In [ ]:
modelWLS.summary()

In [ ]:
#attempted to use reverse geocode, but met with GeoCoderService Error due to too many requests
#alternative method: approximate boundaries for Manhattan and Brookyln using boundingbox
lat_min = 40.694197
lon_max = -73.896398

def borough_sort(lat_lon): #bounding "Manhattan" into a rough box, assuming everything outside "Manhattan" is "BKLYN"
    lat, lon = lat_lon
    if ((lat >= lat_min) & (lon <= lon_max)):
        return 1
    else:
        return 0



In [ ]:
## Reverse geocoding 
#this cells reverse geocodes coordinates to zip codes
county = np.array([''] * len(cb2015))
#my API is saved as an environmental variable
#gAPI = os.getenv('GOOGLEAPI')
#my API saved in a local file as gAPI = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX'
from gAPI import gAPI
for i,ll in enumerate(pd.DataFrame(latlon).drop_duplicates().values):
    print ("%d/%d"%(i, nstations))
    url = ("https://maps.googleapis.com/maps/api/geocode/json?latlng=" +
           "%f,%f&key=%s"%(
            ll[0], ll[1], gAPI))
    #print ("https://maps.googleapis.com/maps/api/geocode/json?latlng=" +
    #       "%f,%f&key=%s"%(
    #        ll[0], ll[1], os.getenv('GOOGLEAPI')))
    #print (get_jsonparsed_data(url)["results"][0]['address_components'][4])
    # for testing break loop after N
    #if i>10: break
    revgeo = get_jsonparsed_data(url)["results"][0]['address_components'][4]
    #print (revgeo)
    county[(latlon[:,0] == ll[0]) * 
           (latlon[:,1] == ll[1])] = revgeo['long_name']
cb2015['county'] = county
cb2015.head()


https://github.com/rxl204/PUI2017_rxl204/blob/master/HW4_rxl204/Assignment4.ipynb

In [ ]:
## fitting a line
p1, p0, linmodel_1 = fit_line1(np.log10(bblnrgdataCut.UnitsTotal), 
                        np.log10(bblnrgdataCut.nrg))
pl.figure(figsize=(10,10))
pl.scatter(np.log10(bblnrgdataCut.UnitsTotal), np.log10(bblnrgdataCut.nrg))
plot(np.log10(bblnrgdataCut.UnitsTotal), linmodel_1.predict(), 'k')
xl = pl.xlabel("log10 Number of Units in Building", fontsize=20)
yl = pl.ylabel("building log10 Energy consumption per (kBtu)", fontsize=20)
linmodel_1.summary()

In [ ]:
## polynomial fit
#I find the easiest way to use the formula package is to use a dataframe 
#with the quantities that are not linear already calculated

bblnrgdataCut['logNrg']  = np.log10(bblnrgdataCut.nrg)
bblnrgdataCut['logUnits']  = np.log10(bblnrgdataCut.UnitsTotal)

X = np.linspace(bblnrgdataCut['logUnits'].min(), bblnrgdataCut['logUnits'].max(), 1000)
curvemodel = smf.ols(formula = 'logNrg ~ logUnits + I(logUnits**2)', 
                          data = bblnrgdataCut).fit()
pl.figure(figsize=(16,14))
pl.scatter(np.log10(bblnrgdataCut.UnitsTotal), np.log10(bblnrgdataCut.nrg))
plot(X, curvemodel.predict(exog = dict(logUnits = X)), 'k')
xl = pl.xlabel("log10 Number of Units in Building", fontsize=20)
yl = pl.ylabel("building log10 Energy consumption per (kBtu)", fontsize=20)
curvemodel.summary()

https://github.com/fedhere/PUI2017_fb55/blob/master/HW6_fb55/building_nrg_solution.ipynb

In [ ]:
#import mpld3
fig, ax = plt.subplots()

scatter = ax.scatter(df_.gdp * 1e-12, df_['Number of mass shootings'])
ax.errorbar(df_.gdp * 1e-12, df_['Number of mass shootings'], 
            yerr = np.sqrt(df_['Number of mass shootings'] * 1.0), fmt = '.')
ax.set_xlabel("GDP (in tillion $)")
ax.set_ylabel("Number of mass shooting per person")
ax.set_ylim();
#labels = ['{0}, {1}'.format(c, n) for c,n in zip(df_["Country"].values, 
#                                                 df_["Number of mass shootings"].values)]
#tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=labels)
#mpld3.plugins.connect(fig, tooltip)
#mpld3.display()

In [ ]:
#checking for outliers
df_[df_.gdp > 10e12]

In [ ]:
df_[df_["Number of mass shootings"] > 10]